In [50]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_classic.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.storage import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_classic.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1200,
    chunk_overlap=200,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt=ChatPromptTemplate.from_messages([
    ("system","""
     You will receive a chunk of text.\n
     If it contains information that helps answer the question, 
     extract the minimal relevant quote(s) verbatim.\n
     If not relevant, return an empty string.\n\n
     CHUNK:        
    ------
    {context}
    """),
    ("human","{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    results = []
    for doc in documents:
        result = map_doc_chain.invoke({
        "context": doc.page_content,
        "question": question
    }).content
        results.append(result)

    return "\n\n".join(results)

map_chain = {"documents":retriever, "question":RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", 
    """
    Given the following extracted parts of a long document and a question and chat history, create a final answer. If you don't know the answer, just sat that you don't know. Don't try to make up an answer.
    
    ------
    {context}
    """,),MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"context": map_chain, "question": RunnablePassthrough(), "history":load_memory} | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"ouput":result.content})
    print(result)

In [ ]:

invoke_chain("What message did he write on the table? quote it exactly.")

content='The message he wrote on the table was: "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 131, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Crpf7LGY6H3dfbArGjts1CznFLaUH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b6630-7080-7280-a78a-9163b47ecf1f-0' usage_metadata={'input_tokens': 131, 'output_tokens': 29, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [55]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What message did he write on the table? quote it exactly.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The message he wrote on the table was: "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE"', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Is Aaronson guilty?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is Julia?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Julia is a character in the text who is having a conversation with another character.', additional_kwargs={}, response_metadata={})]}

In [53]:
invoke_chain("Is Aaronson guilty?")


content='Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 235, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Crpg0uHY2OATjN8AAbiTDhFJbeJNj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b6631-4582-7371-ba72-883951bc1eae-0' usage_metadata={'input_tokens': 235, 'output_tokens': 26, 'total_tokens': 261, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [54]:
invoke_chain("Who is Julia?")


content='Julia is a character in the text who is having a conversation with another character.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 266, 'total_tokens': 283, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CrpgQLhxRbeHDWHRHiCMyt1fivWDX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b6631-ab7a-73f0-8357-69d19bf7d9f5-0' usage_metadata={'input_tokens': 266, 'output_tokens': 17, 'total_tokens': 283, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
